# 🚀 Meta-Llama3-8b-Instruct CoT Agent with vLLM

![](https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fcot.1933d9fe.png&w=1920&q=75)

LLM本质上是自回归模型，下一个token预测依赖之前的上下文，通过给予一些token空间让模型对问题做进一步拆在，一步步思考最后得到结论，可以显著提升模型解决复杂问题的准确率，这就CoT(Chain of Thoughts)的核心。在[mistral-CoT-Agent](./mistral-CoT-Agent.ipynb)案例中，我们利用GBNF对语言模型输出进行了强制规范，这次我们测试一下`Llama3-8b-Instruct`模型的能力：
1. 是否能够自然的支持CoT场景
2. 对中文prompt的支持能力

In [2]:
from vllm import LLM, SamplingParams

llm = LLM(
    model="/data/hf/Meta-Llama-3-8B-Instruct",
    trust_remote_code=True,
    tensor_parallel_size=2,
)
tokenizer = llm.get_tokenizer()

/home/kky/miniconda3/envs/gomars/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-19 22:40:58,774	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 04-19 22:40:58 config.py:407] Custom all-reduce kernels are temporarily disabled due to stability issues. We will re-enable them once the issues are resolved.


2024-04-19 22:41:00,727	INFO worker.py:1724 -- Started a local Ray instance.


INFO 04-19 22:41:01 llm_engine.py:79] Initializing an LLM engine with config: model='/data/hf/Meta-Llama-3-8B-Instruct', tokenizer='/data/hf/Meta-Llama-3-8B-Instruct', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=2, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 04-19 22:41:12 llm_engine.py:337] # GPU blocks: 12465, # CPU blocks: 4096
INFO 04-19 22:41:14 model_runner.py:666] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-19 22:41:14 model_runner.py:670] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
(RayWorkerVllm pid=263474) INFO 04-19 22:41:14 model_runner.py:666] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(RayWorkerVllm pid=263474) INFO 04-19 22:41:14 model_runner.py:670] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are r

(RayWorkerVllm pid=263474) INFO 04-19 22:41:20 model_runner.py:738] Graph capturing finished in 6 secs.


In [6]:
question = "自助餐厅有23个苹果。 如果他们用 20 个苹果做午餐，然后又买了 6 个，他们有多少个苹果？"
system_prompt = '''你是一个卓越的AI助手，一步步思考回答用户的问题，回答格式遵循：
思考：将你的思考过程写在这里。
回答：将你的最终答案放在这里。
'''

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question}
]
prompt = tokenizer.apply_chat_template(messages,tokenize=False,)

In [7]:
outputs = llm.generate(
    [prompt],
    SamplingParams(
        temperature=0.,
        # do_sample=False,
        top_p=0.9,
        max_tokens=1024,
        stop_token_ids=[tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")],  # KEYPOINT HERE
    )
)[0].outputs[0].text

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.29s/it]


In [8]:
print(outputs)

思考：

首先，我们需要知道自助餐厅原来有23个苹果。然后，他们用20个苹果做午餐，这意味着他们已经使用了20个苹果。剩下的苹果数量是23 - 20 = 3个。

接着，他们又买了6个苹果，这意味着他们现在拥有3 + 6 = 9个苹果。

回答：

自助餐厅现在有9个苹果。


完全正确，格式满足要求，同时对中文prompt的理解以及中文输出能力都在水准以上。